### 1

### 재무제표 전처리

### 정리된 코드

In [ ]:
# ================================
# 라이브러리 설치 (노트북 환경 가정)
# ================================
!pip install dart_fss

# ================================
# 임포트
# ================================
import os                      # 환경변수에서 API 키를 읽기 위해 사용
from pathlib import Path
import math
import re
from typing import List, Optional, Dict

import numpy as np
import pandas as pd
import dart_fss as dart        # DART 비공식 Python 라이브러리
import dart_fss


# ================================
# 1) API 키 설정
# ================================
API_KEY = os.getenv("DART_API_KEY", "66ce66618f4850247aa36d3d0bea34737980af17").strip()  # 환경변수에서 키를 읽음(없으면 기본 문자열)
if not API_KEY:                                   # 키가 비어 있으면,
    API_KEY = "인증키 입력"                       # 직접 문자열로 넣는 방법(임시)
dart.set_api_key(api_key=API_KEY)                 # dart-fss에 키를 등록해야 API 호출 가능

# ================================

# 2) 회사 찾기
# ================================
corp_list = dart.get_corp_list()                  # 전체 회사 목록(오프라인 캐시됨)
cn = input("찾으시는 회사명을 입력하세요")
# corp = corp_list.find_by_stock_code("005930")   # 종목코드로 삼성전자 찾기(예: 005930)
corp = corp_list.find_by_corp_name(cn, exactly=True)[0]  # 회사명으로 찾는 방법

# ================================
# 3) 재무제표 추출(extract_fs)
#   - bgn_de: 시작일(YYYYMMDD). 이 날짜 이후 공시들을 기준으로 재무제표 구성
#   - report_tp: 'annual'(연간), 'half'(반기), 'quarter'(분기)
#   - separate: False=연결, True=개별
# ================================
fs = corp.extract_fs(
    bgn_de="20190101",       # 2018년 1월 1일부터
    report_tp="annual",      # 연간 보고서(사업보고서)
    separate=False,          # 연결 재무제표(일반적으로 이걸 많이 씁니다.)
    progressbar=False
)
# fs.save(filename="연결_연간.xlsx")

# ================================
# 4) 유틸 함수
# ================================
# 4-1) 안전한 나눗셈(0/NaN 방지)
def safe_div(a, b):
    """a/b 계산 시 0 나누기, NaN 발생을 방지하고 NaN을 깔끔히 반환"""
    if a is None or b is None:
        return float('nan')
    if pd.isna(a) or pd.isna(b) or b == 0:
        return float('nan')
    return a / b

# 4-2) 특정 계정값을 읽어오는 도우미 (라벨 열 이름을 유연하게)
#    - df_flat: flatten_statement() 결과
#    - keywords: 포함 검색 키워드
#    - col: 'current', 'prev', 'prev2' ...
#    - label_col: 라벨 열 이름 (기본 후보: label, 계정, 항목, account)
def pick_value(df_flat, keywords, col='current', label_col=None):
    if df_flat is None or len(df_flat) == 0:
        return float('nan')  # 표 자체가 없으면 NaN

    # 라벨 열 자동 추정
    if label_col is None:
        candidates = ['label', '계정', '항목', 'account', 'name']
        label_col = next((c for c in candidates if c in df_flat.columns), None)

    if label_col is None:
        # 라벨 열을 못 찾으면 첫 번째 열을 사용 (마지막 방어)
        label_col = df_flat.columns[0]

    # 불리언 마스크(시리즈) 준비
    mask = pd.Series(False, index=df_flat.index)

    # 키워드 포함 검색 (대소문자 무시, NaN 안전)
    for kw in keywords:
        mask = mask | df_flat[label_col].astype(str).str.contains(kw, case=False, na=False)

    hits = df_flat.loc[mask]
    if hits.empty or col not in df_flat.columns:
        return float('nan')  # 못 찾거나 해당 값 열이 없으면 NaN

    # 가장 위(첫 행)의 지정 열 값을 숫자로 변환 후 반환
    val = pd.to_numeric(hits.iloc[0][col], errors='coerce')
    return float(val) if not pd.isna(val) else float('nan')
# ================================
# 5) 원본 표 가져오기
# ================================
cis_flat = fs['cis']   # 포괄손익계산서
is_flat  = fs['is']    # 포괄손익계산서
bs_flat  = fs['bs']    # 재무상태표
cf_flat  = fs['cf']    # 현금흐름표

# cond = bs_flat["영업이익"]  
from pandas.api.types import is_numeric_dtype

# ================================
# 6) 전처리
# ================================
# 6-1) 손익계산서(is)
if "is_flat" in globals() and isinstance(is_flat, pd.DataFrame) and not is_flat.empty:
    is_flat = is_flat.T.copy()

    if isinstance(is_flat.columns, pd.MultiIndex):
        is_flat.columns = is_flat.columns.get_level_values(-1)

    if is_flat.shape[0] > 1:
        is_flat.columns = is_flat.iloc[1]

    if is_flat.shape[0] >= 6:
        is_flat = is_flat.drop(is_flat.index[0:6])

    if getattr(is_flat.index, "nlevels", 1) > 1:
        is_flat = is_flat.reset_index(level=1, drop=True)

    is_flat.index = is_flat.index.astype(str).str.split("-").str[-1]

# 6-2) 재무상태표(bs)
bs_flat = bs_flat.T
bs_flat.columns = bs_flat.columns.get_level_values(-1)
bs_flat.columns = bs_flat.iloc[1]
bs_flat = bs_flat.reset_index(level=1, drop=True)
cond = bs_flat.index.to_series().astype(str).str.isdigit()
bs_flat = bs_flat.loc[cond]
bs_flat.index = bs_flat.index.astype(str).astype(int)

# 6-3) 포괄손익계산서(cis)
cis_flat = cis_flat.T
cis_flat.columns = cis_flat.columns.get_level_values(-1)
cis_flat.columns = cis_flat.iloc[1]
cis_flat = cis_flat.drop(cis_flat.index[0:7])
cis_flat = cis_flat.reset_index(level=1, drop=True)
cis_flat.index = cis_flat.index.str.split('-').str[-1]  # 인덱스 이름 통일



# ================================
# 1) 별칭 사전
# ================================
KOR_KEY_ALIASES = {
    # =========================
    # BS (재무상태표)
    # =========================
    "current_assets": [
        "유동자산", "유동 자산", "유동자산계", "총유동자산", "유동자산 총계", "유동자산합계", "유동자산금액"
    ],
    "current_liabilities": [
        "유동부채", "유동 부채", "유동부채계", "총유동부채", "유동부채 총계", "유동부채합계"
    ],
    "noncurrent_liabilities": [
        "비유동부채", "비유동 부채", "장기부채", "비유동부채계", "비유동부채 총계", "비유동부채합계"
    ],
    "total_liabilities": [
        "부채총계", "총부채", "부채 총계", "부채합계", "부채총액", "부채 계", "부채의총계"
    ],
    "equity_total": [
        "자본총계", "자본 총계", "총자본", "자본합계", "자본총액", "자본 계", "자본의총계"
    ],
    # 지배/비지배지분(다양한 표기)
    "equity_parent": [
        "지배기업주주지분", "지배기업 소유지분", "지배기업소유주지분", "지배기업 소유주지분"
    ],
    "equity_nci": [
        "비지배지분", "비지배 지분", "비지배주주지분", "소수주주지분", "비지배지분(자본)"
    ],
    "total_assets": [
        # 자산총계 계열
        "자산총계", "자산 총계", "자산총액", "총자산", "자산 합계", "자산합계", "자산계", "자산의총계", "자산계정합계",
        # '부채와 자본 총계' 계열(동일 총계 라인을 이렇게 표기하는 회사 포함)
        "부채와자본총계", "부채와 자본 총계", "부채 및 자본 총계", "자본과부채총계", "자본과 부채 총계", "부채자본총계", "부채와자본 합계"
    ],

    # =========================
    # PL (IS/CIS)
    # =========================
    "revenue": [
        "매출액", "수익(매출액)", "영업수익"
    ],
    "revenue_prev": [
        "매출액(전년도)", "전년도매출"
    ],
    "operating_income": [
        "영업이익", "영업이익(손실)", "영업손실", "영업손익"
    ],
    "operating_income_preLLP": [
        "신용손실충당금 반영전 영업이익"
    ],
    "credit_loss": [
        "신용손실충당금 전입액", "대손상각비"
    ],
    "finance_costs": [
        "금융비용", "금융원가", "이자비용", "보험금융이자비용"
    ],

    # 수익(근사) 구성요소 (없을 때만 합산 사용)
    "interest_income": [
        "이자수익", "보험금융이자수익",
        "기타포괄손익-공정가치 측정 및 상각후원가 측정 금융상품 이자수익",
        "당기손익-공정가치로 측정하는 금융자산의 이자수익"
    ],
    "fee_income": [
        "수수료수익"
    ],
    "insurance_revenue": [
        "보험수익", "재보험수익"
    ],
}

# ▶ 삼성전자 포괄손익 관련 “그대로 보여줄” 패스스루 컬럼 목록
SAMSUNG_EXTRA_PASSTHROUGH = [
    "당기순이익",
    "기타포괄손익",
    "후속적으로 당기손익으로 재분류되지 않는 포괄손익",
    "기타포괄손익-공정가치금융자산평가손익",
    "관계기업 및 공동기업의 기타포괄손익에 대한 지분",
    "순확정급여부채(자산) 재측정요소",
    "후속적으로 당기손익으로 재분류되는 포괄손익",
    "해외사업장환산외환차이",
    "현금흐름위험회피파생상품평가손익",
    "총포괄손익",
]

# ================================
# 2) 인덱스/이름 정규화 유틸
# ================================
def _normalize_name(s: str) -> str:
    """공백/특수공백/기호 제거 + 소문자화"""
    s = str(s)
    # 보이지 않는 공백 제거
    s = s.replace("\u00A0", "").replace("\u200B", "").replace("\u2009", "").replace("\u202F", "")
    return re.sub(r'[\s\(\)\-_,/·\.]', '', s).casefold()

def _normalize_date_index_like(idx_val: str) -> str:
    """
    'yyyymmdd' 또는 'yyyymmdd-yyyymmdd' → 뒤쪽 yyyymmdd만 보존
    그 외는 숫자만 추출해서 앞 8자리
    """
    s = str(idx_val)
    if "-" in s:
        s = s.split("-")[-1]
    digits = re.sub(r"\D", "", s)
    return digits[:8] if len(digits) >= 8 else s  # 최소 방어

def _unify_df_index_yyyymmdd(df: pd.DataFrame) -> pd.DataFrame:
    """인덱스를 yyyymmdd 문자열로 통일, 중복은 마지막값 유지, 오름차순 정렬"""
    if not isinstance(df, pd.DataFrame) or df.empty:
        return df
    new_idx = [_normalize_date_index_like(x) for x in df.index]
    out = df.copy()
    out.index = pd.Index(new_idx, dtype="object")
    out = out[~out.index.duplicated(keep="last")]
    out = out.sort_index()
    return out

# ================================
# 3) 컬럼 탐색/숫자 변환 유틸
# ================================
def _pick_first_exist(cols: List[str], aliases: List[str]) -> Optional[str]:
    """정확일치 → 정규화일치 → 부분일치 순으로 첫 매칭 반환"""
    cols_str = [str(c) for c in cols]
    for a in aliases:
        if a in cols_str:
            return a
    colmap = {_normalize_name(c): c for c in cols_str}
    for a in aliases:
        na = _normalize_name(a)
        if na in colmap:
            return colmap[na]
    for a in aliases:
        na = _normalize_name(a)
        for nk, orig in colmap.items():
            if na in nk:
                return orig
    return None

def _to_numeric_safe_series(s: pd.Series) -> pd.Series:
    s2 = s.astype(str).str.strip()
    s2 = s2.str.replace('%', '', regex=False)
    s2 = s2.str.replace(',', '', regex=False)
    s2 = s2.str.replace(r'^\((.*)\)$', r'-\1', regex=True)  # 괄호표기 음수 → -부호
    return pd.to_numeric(s2, errors="coerce")

def _resolve_numeric_series(df: pd.DataFrame, colname: str) -> pd.Series:
    obj = df.loc[:, colname]
    if isinstance(obj, pd.DataFrame):
        tmp = obj.apply(_to_numeric_safe_series, axis=0)
        return tmp.sum(axis=1, skipna=True)
    return _to_numeric_safe_series(obj)

def _get_colname(df: Optional[pd.DataFrame], key: str) -> Optional[str]:
    if not isinstance(df, pd.DataFrame) or df.empty:
        return None
    return _pick_first_exist(df.columns.tolist(), KOR_KEY_ALIASES[key])

def _get_series(df: Optional[pd.DataFrame], key: str) -> Optional[pd.Series]:
    if not isinstance(df, pd.DataFrame) or df.empty:
        return None
    name = _get_colname(df, key)
    if name is None:
        return None
    return _resolve_numeric_series(df, name)

def _sum_all_aliases(df: Optional[pd.DataFrame], aliases: List[str]) -> Optional[pd.Series]:
    """여러 별칭을 모두 찾아 합산(있을 때만)"""
    if not isinstance(df, pd.DataFrame) or df.empty:
        return None
    cols = df.columns.tolist()
    present = []
    # 정확일치
    for a in aliases:
        present += [c for c in cols if c == a]
    # 정규화일치
    norm_map = {_normalize_name(c): c for c in cols}
    for a in aliases:
        na = _normalize_name(a)
        if na in norm_map:
            present.append(norm_map[na])
    # 부분일치
    for a in aliases:
        na = _normalize_name(a)
        for nk, orig in norm_map.items():
            if na in nk:
                present.append(orig)
    # 중복 제거 순서 유지
    present = list(dict.fromkeys(present))
    if not present:
        return None
    total = None
    for name in present:
        ser = _resolve_numeric_series(df, name)
        total = ser if total is None else total.add(ser, fill_value=0.0)
    return total

def _ensure_series(x, index: pd.Index) -> pd.Series:
    """항상 지정 index 로 맞춰진 Series를 반환"""
    if isinstance(x, pd.Series):
        return x.reindex(index)
    return pd.Series(np.nan, index=index, dtype=float)

def _safe_div(a: Optional[pd.Series], b: Optional[pd.Series], index: Optional[pd.Index] = None) -> pd.Series:
    if a is None and b is None:
        return _ensure_series(None, (index if index is not None else pd.Index([], dtype="object")))
    if a is None:
        return _ensure_series(None, getattr(b, "index", index))
    if b is None:
        return pd.Series(np.nan, index=getattr(a, "index", index), dtype=float)
    out = a.astype(float).divide(b.astype(float))
    return out.replace([np.inf, -np.inf], np.nan)

YEAR_RE = re.compile(r'(19|20)\d{2}')
def _prev_by_year_series(r: Optional[pd.Series]) -> Optional[pd.Series]:
    """연도말 기준 이전 연도의 값(전년도 매출 등) 추정"""
    if not isinstance(r, pd.Series) or r.empty:
        return None
    idx_str = r.index.astype(str)
    years = idx_str.map(lambda s: (YEAR_RE.search(s).group(0) if YEAR_RE.search(s) else None))
    if years.isna().all():
        return None

    def _to_order_val(x: str):
        digits = re.sub(r'[^0-9]', '', x or '')
        if len(digits) >= 8:
            v = digits[-8:]
            try:
                return pd.to_datetime(v, format="%Y%m%d", errors="raise")
            except Exception:
                pass
        try:
            return pd.to_datetime(x, errors="coerce")
        except Exception:
            return pd.NaT

    order = idx_str.map(_to_order_val)
    if order.isna().all():
        order = idx_str

    df_tmp = pd.DataFrame({"y": years, "v": pd.to_numeric(r, errors="coerce"), "_order": order}, index=r.index)
    df_tmp = df_tmp.dropna(subset=["y"]).sort_values("_order")
    year_last = df_tmp.groupby("y")["v"].last()

    prev_year = df_tmp["y"].astype(int).sub(1).astype(str)
    rp_vals = prev_year.map(year_last)
    rp_vals.index = df_tmp.index

    rp = pd.Series(np.nan, index=r.index, dtype=float)
    rp.loc[rp_vals.index] = rp_vals.values
    return rp

# ================================
# 4) 메인 계산 함수
# ================================
def compute_metrics_df_flat_kor(
    bs_flat_df: Optional[pd.DataFrame],
    is_flat_df: Optional[pd.DataFrame] = None,
    cis_flat_df: Optional[pd.DataFrame] = None,
    key_cols: Optional[List[str]] = None,  # 현재는 미사용
) -> pd.DataFrame:

    # (A) 기본 방어
    if not isinstance(bs_flat_df, pd.DataFrame) or bs_flat_df.empty:
        cols = [
            "유동자산","유동부채","자산총계","부채총계",
            "매출액","매출액(전년도)","영업이익","금융비용",
            "유동비율","부채비율","이자보상배수","영업이익률","매출증가율"
        ]
        return pd.DataFrame(columns=cols)

    # 인덱스 통일(yyyymmdd 문자열)
    bs = _unify_df_index_yyyymmdd(bs_flat_df.copy())
    is_pl_df  = _unify_df_index_yyyymmdd(is_flat_df.copy())  if isinstance(is_flat_df,  pd.DataFrame) and not is_flat_df.empty  else None
    cis_pl_df = _unify_df_index_yyyymmdd(cis_flat_df.copy()) if isinstance(cis_flat_df, pd.DataFrame) and not cis_flat_df.empty else None

    # (B) BS 컬럼 식별 + 숫자화/해소
    ca  = _get_series(bs, "current_assets")          # 유동자산
    cl  = _get_series(bs, "current_liabilities")     # 유동부채
    ncl = _get_series(bs, "noncurrent_liabilities")  # 비유동부채
    tl  = _get_series(bs, "total_liabilities")       # 부채총계
    eqt = _get_series(bs, "equity_total")            # 자본총계
    eqp = _get_series(bs, "equity_parent")           # 지배기업 소유주지분
    eqn = _get_series(bs, "equity_nci")              # 비지배지분
    ta  = _get_series(bs, "total_assets")            # 자산총계(또는 '부채와자본총계')

    # 총부채 없음 → 유동+비유동 합
    if tl is None and (cl is not None or ncl is not None):
        basis_idx = getattr(cl, "index", getattr(ncl, "index", bs.index))
        tl = pd.Series(0.0, index=basis_idx, dtype=float)
        if cl is not None:  tl = tl.add(cl,  fill_value=0.0)
        if ncl is not None: tl = tl.add(ncl, fill_value=0.0)

    # 자본총계 우선, 없으면 (부모+비지배), 그것도 없으면 자산-부채
    if eqt is not None:
        eq = eqt
    elif (eqp is not None) and (eqn is not None):
        eq = eqp.add(eqn, fill_value=0.0)
    elif (ta is not None) and (tl is not None):
        eq = ta.sub(tl, fill_value=np.nan)
    else:
        eq = None

    # (C) PL 소스: IS 우선, 없으면 CIS; 항목별 보충
    def _extract_pl(df):
        if not isinstance(df, pd.DataFrame) or df.empty:
            return (None, None, None, None, None, None, df)
        r   = _get_series(df, "revenue")
        rp  = _get_series(df, "revenue_prev")
        oi  = _get_series(df, "operating_income")
        oi0 = _get_series(df, "operating_income_preLLP")
        clo = _get_series(df, "credit_loss")
        fc  = _get_series(df, "finance_costs")
        return (r, rp, oi, oi0, clo, fc, df)

    r1, rp1, oi1, oi01, clo1, fc1, is_pl  = _extract_pl(is_pl_df)
    r2, rp2, oi2, oi02, clo2, fc2, cis_pl = _extract_pl(cis_pl_df)

    # 우선순위(항목별 보충)
    r   = r1   if r1  is not None else r2
    rp  = rp1  if rp1 is not None else rp2
    oi  = oi1  if oi1 is not None else oi2
    oi0 = oi01 if oi01 is not None else oi02
    clo = clo1 if clo1 is not None else clo2
    fc  = fc1  if fc1 is not None else fc2

    # 대표 PL df
    pl_any = is_pl if (isinstance(is_pl, pd.DataFrame) and not is_pl.empty) else cis_pl

    # ① 수익(근사): 없을 때만 이자수익+수수료수익+보험수익 합산
    if r is None and isinstance(pl_any, pd.DataFrame):
        interest = _sum_all_aliases(pl_any, KOR_KEY_ALIASES["interest_income"])
        fee      = _sum_all_aliases(pl_any, KOR_KEY_ALIASES["fee_income"])
        ins      = _sum_all_aliases(pl_any, KOR_KEY_ALIASES["insurance_revenue"])
        parts = [p for p in [interest, fee, ins] if isinstance(p, pd.Series)]
        if parts:
            total = parts[0]
            for p in parts[1:]:
                total = total.add(p, fill_value=0.0)
            r = total

    # ② 영업이익 보충: '신용손실충당금 반영전 영업이익' - '신용손실충당금 전입액'
    if oi is None and oi0 is not None:
        oi = oi0 if clo is None else oi0.sub(clo, fill_value=0.0)

    # ③ 전년도 매출 Fallback(연도 정규식)
    if (rp is None) or (not isinstance(rp, pd.Series)) or rp.isna().all():
        rp = _prev_by_year_series(r)

    # (D) 공통 인덱스(문자열) 구성
    union_idx = pd.Index(bs.index.astype(str), dtype="object")
    for s in [r, rp, oi, fc, ca, cl, ta, tl, eq]:
        if isinstance(s, pd.Series):
            union_idx = union_idx.union(pd.Index(s.index.astype(str), dtype="object"))

    def _on_idx(s):
        return _ensure_series(s, union_idx)

    ca = _on_idx(ca); cl = _on_idx(cl); ta = _on_idx(ta); tl = _on_idx(tl); eq = _on_idx(eq)
    r  = _on_idx(r);  rp = _on_idx(rp); oi = _on_idx(oi);  fc = _on_idx(fc)

    # eq 전부 NaN이면 최종 보정
    if isinstance(eq, pd.Series) and eq.isna().all() and (ta is not None) and (tl is not None):
        eq = ta - tl

    # (E) 지표 계산 (비율은 '배' 기준, 표시할 때 % 변환 권장)
    current_ratio     = _safe_div(ca, cl, union_idx)     # 유동자산 / 유동부채
    debt_ratio        = _safe_div(tl, eq, union_idx)     # 부채총계 / 자본총계
    interest_coverage = _safe_div(oi, fc, union_idx)     # 영업이익 / 금융비용
    op_margin         = _safe_div(oi, r,  union_idx)     # 영업이익 / 매출액
    sales_growth      = _safe_div(r - rp, rp, union_idx) # (매출 - 전년) / 전년

    # (F) 결과 테이블
    out = pd.DataFrame(index=union_idx)
    out.index.name = "date"
    out["유동자산"]       = ca
    out["유동부채"]       = cl
    out["자산총계"]       = ta
    out["부채총계"]       = tl
    out["매출액"]         = r
    out["매출액(전년도)"] = rp
    out["영업이익"]       = oi
    out["금융비용"]       = fc
    out["유동비율"]       = current_ratio
    out["부채비율"]       = debt_ratio
    out["이자보상배수"]   = interest_coverage
    out["영업이익률"]     = op_margin
    out["매출증가율"]     = sales_growth

    # ▶ (추가) BS: 지배/비지배지분도 함께 출력(있을 때만)
    if isinstance(eqp, pd.Series):
        out["지배기업 소유주지분"] = _on_idx(eqp)
    if isinstance(eqn, pd.Series):
        out["비지배지분"] = _on_idx(eqn)

    # ▶ (추가) PL: 삼성전자 포괄손익 관련 컬럼들을 그대로 패스스루(있을 때만)
    if isinstance(pl_any, pd.DataFrame) and not pl_any.empty:
        for col in SAMSUNG_EXTRA_PASSTHROUGH:
            ser = _sum_all_aliases(pl_any, [col])  # 정규화/부분일치까지 케어
            if isinstance(ser, pd.Series):
                out[col] = _on_idx(ser)

    # 날짜 내림차순 정렬
    try:
        out = out.sort_index(ascending=False, key=lambda s: pd.to_datetime(s, format="%Y%m%d", errors="coerce"))
    except Exception:
        out = out.sort_index(ascending=False)
    return out

# ================================
# 5) 예시 실행부 (환경에 변수 있을 때만)
# ================================
if "bs_flat" in globals():
    result_df = compute_metrics_df_flat_kor(
        bs_flat_df=bs_flat,
        is_flat_df=(is_flat if "is_flat" in globals() else None),
        cis_flat_df=(cis_flat if "cis_flat" in globals() else None),
        key_cols=None,
    )

    # 보고용 컬럼 순서
    cols_order = [
        "유동자산","유동부채","자산총계","부채총계",
        "매출액","매출액(전년도)","영업이익","금융비용",
        "유동비율","부채비율","이자보상배수","영업이익률","매출증가율",
        # "당기순이익","기타포괄손익","총포괄손익",  # 필요 시 추가
    ]
    to_show = result_df.loc[:, [c for c in cols_order if c in result_df.columns]].copy()

    # 최신 기간 1건 보기 + 포맷팅(비율 → %)
    def format_table(df: pd.DataFrame) -> pd.DataFrame:
        df2 = df.copy()
        pct_cols = ["유동비율","부채비율","영업이익률","매출증가율"]
        for c in pct_cols:
            if c in df2.columns:
                df2[c] = (df2[c] * 100).round(2).astype(str) + "%"
        if "이자보상배수" in df2.columns:
            df2["이자보상배수"] = df2["이자보상배수"].round(2)
        return df2

    # 인덱스 날짜 정렬(오름차순) 후 마지막 1개
    tmp = to_show.copy()
    tmp["_period"] = pd.to_datetime(tmp.index.astype(str), format="%Y%m%d", errors="coerce")
    tmp = tmp.sort_values("_period").drop(columns="_period")
    to_show_latest = tmp.tail(1)

    print(format_table(to_show_latest))
